In [32]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}

    {example}

    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]

full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)

chain = full_prompt | chat

chain.invoke({
    "character": "Pirate",
    "example_question": "What is your location?",
    "example_answer": "Arrrg! That is a secret!! Arg arg!!",
    "question": "What is your fav food?",
})

Arrr matey! Me favorite grub be a hearty plate o' salted beef and hardtack! Aye, nothin' like a good meal to keep me strength up on the high seas! Arrr!

AIMessageChunk(content="Arrr matey! Me favorite grub be a hearty plate o' salted beef and hardtack! Aye, nothin' like a good meal to keep me strength up on the high seas! Arrr!")